## 5.3 Resnet Pretrained Model - Pytorch Hub

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-Image-Classification/blob/main/Pertemuan%205/5.3%20pytorch_hub_resnet.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="resource/cl-connect-gpu.png" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="resource/cl-connect-gpu-success.png" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

<br><br><br><br>
#### 5.3.1 Intro Pytorch Hub
<img src="resource/PyTorch_logo.png" width="300px" style="background-color:white; padding:10px; border-radius:8px"><br>
- <font color="orange">Discover</font> and <font color="orange">publish models</font> to a pre-trained model repository designed for research exploration.
- Open Pytorch Hub in browser [https://pytorch.org/hub/](https://pytorch.org/hub/)<br>
<img src="resource/torch-hub.png" width="900px">

<br><br><br><br>
#### 5.3.2 ResNet Pretrained Model in Pytorch Hub
- <font color="orange">ResNet</font> - Deep residual networks pre-trained on ImageNet
    - Open in browser [https://pytorch.org/hub/pytorch_vision_resnet/](https://pytorch.org/hub/pytorch_vision_resnet/)
<img src="resource/torch-hub-resnet.png" width="900px">

- Load ResNet model from Pytorch

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

- Download sample image

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

- All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`. 
- The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]` and `std = [0.229, 0.224, 0.225]`.

In [ ]:
from PIL import Image
from torchvision import transforms

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

- move the input and model to GPU for speed if available

In [ ]:
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

- Predict the input tensor `dog.jpg`
    - Output tensor of shape 1000, with confidence scores over ImageNet's 1000 classes

In [ ]:
with torch.no_grad():
    output = model(input_batch)

print(output[0])

- The output has un-normalized scores. 
    - To get probabilities, you can run a <font color="orange">softmax</font> on it.

In [ ]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

- Download ImageNet labels

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

- Read the categories

In [ ]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

- Show top categories per image

In [ ]:
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

### Model Description

Resnet models were proposed in "Deep Residual Learning for Image Recognition".
Here we have the 5 versions of resnet models, which contains 18, 34, 50, 101, 152 layers respectively.
Detailed model architectures can be found in Table 1.
Their 1-crop error rates on ImageNet dataset with pretrained models are listed below.

| Model structure | Top-1 error | Top-5 error |
| --------------- | ----------- | ----------- |
|  resnet18       | 30.24       | 10.92       |
|  resnet34       | 26.70       | 8.58        |
|  resnet50       | 23.85       | 7.13        |
|  resnet101      | 22.63       | 6.44        |
|  resnet152      | 21.69       | 5.94        |

### References

 - [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
 - [Original Notebook from Pytorch Team](https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/pytorch_vision_resnet.ipynb)